In [2]:
import yfinance as yf

class TradingRiskCalculator:
    def __init__(self):
        self.balance = 0
        self.equity = 0
        self.leverage = 0
        self.acc_currency = ""
        self.lot_size = 0
        self.ticker_symbol = ""

    def get_ticker_info(self, symbol):
        if '/' in symbol:
            symbol = symbol.replace("/", "") + "=X"
        try:
            data = yf.Ticker(symbol)
            price = data.history(period="1d")['Close'][0]
            return price
        except Exception:
            return None

    def calculate_forex_money_required(self, lot_size, pair_value, acc_currency, pair_symbol):
        money_required_in_pair_currency = lot_size * 100000 * pair_value
        if acc_currency in pair_symbol:
            return money_required_in_pair_currency
        exchange_rate = self.get_ticker_info(f"{pair_symbol[:3]}/{acc_currency}")
        return money_required_in_pair_currency * exchange_rate if exchange_rate else None

    def calculate_stock_money_required(self, shares, price):
        return shares * price

    def calculate_projections(self, initial_equity, position_size):
        scenarios = {
            "5% Loss": -0.05,
            "10% Loss": -0.10,
            "20% Loss": -0.20,
            "5% Gain": 0.05,
            "10% Gain": 0.10,
            "20% Gain": 0.20
        }
        return {scenario: initial_equity + position_size * pct_change for scenario, pct_change in scenarios.items()}

    def run(self):
        print("Trading Risk Calculator")
        self.balance = float(input("Enter Account Balance: "))
        self.equity = float(input("Enter Account Equity: "))
        self.leverage = float(input("Enter Account Leverage: "))
        self.acc_currency = input("Enter Account Currency: ").upper()
        self.lot_size = float(input("Enter Lot Size: "))
        self.ticker_symbol = input("Enter Ticker Symbol: ").upper()

        current_price = self.get_ticker_info(self.ticker_symbol)
        if not current_price:
            print("Failed to retrieve ticker information.")
            return

        if "/" in self.ticker_symbol:
            money_required = self.calculate_forex_money_required(self.lot_size, current_price, self.acc_currency, self.ticker_symbol)
        else:
            shares = self.lot_size * 100
            money_required = self.calculate_stock_money_required(shares, current_price)

        if money_required is None:
            print("Error in calculating money required.")
            return

        margin_required = money_required / self.leverage
        projections = self.calculate_projections(self.equity, money_required)

        print("\nCALCULATION RESULTS")
        print(f"Balance: {self.balance:,.2f} {self.acc_currency}")
        print(f"Equity: {self.equity:,.2f} {self.acc_currency}")
        print(f"Margin Required: {margin_required:,.2f} {self.acc_currency}")
        print(f"Free Margin: {self.equity - margin_required:,.2f} {self.acc_currency}")
        print(f"Price ({self.ticker_symbol}): {current_price:.4f}")

        print("\nRISK PROJECTIONS")
        for scenario, new_equity in projections.items():
            print(f"{scenario}: {new_equity:,.2f}")

        if margin_required > self.equity:
            print("\n! MARGIN EXCEEDS EQUITY !")
        if margin_required > self.balance * 0.5:
            print("\n! POSITION > 50% BALANCE !")

if __name__ == "__main__":
    calculator = TradingRiskCalculator()
    calculator.run()


Trading Risk Calculator
Enter Account Balance: 100
Enter Account Equity: 90
Enter Account Leverage: 100
Enter Account Currency: usd
Enter Lot Size: 0.01
Enter Ticker Symbol: EUR/USD

CALCULATION RESULTS
Balance: 100.00 USD
Equity: 90.00 USD
Margin Required: 10.29 USD
Free Margin: 79.71 USD
Price (EUR/USD): 1.0294

RISK PROJECTIONS
5% Loss: 38.53
10% Loss: -12.94
20% Loss: -115.89
5% Gain: 141.47
10% Gain: 192.94
20% Gain: 295.89


<ipython-input-2-f80fa16c17e1>:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  price = data.history(period="1d")['Close'][0]
